In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import  Sequential
from tensorflow.keras import layers 
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import seaborn as sns

In [23]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.shape, test.shape

((42000, 785), (28000, 784))

In [24]:
y = train.iloc[:,0].values.astype('int32')
X = train.iloc[:,1:].values.astype('float32')
X_test = test.values.astype('float32')

In [25]:
X = X / 255.0
X_test = X_test / 255.0

X = X.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

y = to_categorical(y)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

generator = ImageDataGenerator(rotation_range=8, 
                               width_shift_range=0.08, 
                               shear_range=0.3,
                               height_shift_range=0.08, 
                               zoom_range=0.08
                              )

#Create batch generators for train and validation sets
train_batches = generator.flow(X_train, y_train, 
                               batch_size = 64)

val_batches = generator.flow(X_val, y_val, 
                             batch_size = 64)

In [27]:
early_stopping = EarlyStopping(min_delta=0.001, 
                               patience=10, 
                               restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [28]:
model2 = Sequential([
    layers.Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    layers.BatchNormalization(),
    layers.Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size = (2,2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'),
    layers.BatchNormalization(),
    layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Dense(10, activation = 'softmax')
])

#Define Optimizer
optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


#Compile Model
model2.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

#Fit Model
history = model2.fit_generator(
    generator=train_batches, 
    steps_per_epoch=525,
    epochs=50,  
    validation_data=val_batches, 
    validation_steps=100,
    callbacks = [learning_rate_reduction]
)

Epoch 1/50
525/525 [==============================] - 604s 1s/step - loss: 0.2328 - accuracy: 0.9260 - val_loss: 0.3759 - val_accuracy: 0.8944
Epoch 2/50
525/525 [==============================] - 554s 1s/step - loss: 0.0813 - accuracy: 0.9749 - val_loss: 0.0546 - val_accuracy: 0.9848
Epoch 3/50
 96/525 [====>.........................] - ETA: 7:05 - loss: 0.0619 - accuracy: 0.9779

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize = (15,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
# plt.yticks(np.linspace(0.96,0.995))

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='lower left')


plt.tight_layout()
plt.show();

In [ ]:
predictions = model2.predict_classes(X_test)
submission_data = pd.DataFrame({'ImageId': range(1,28001), 'Label': predictions})
submission_data.to_csv("submission.csv", index=False, header=True)